## Классификация текстов (SPAM/HAM Document Classification)

Спам-фильтр с помощью библиотеки **scikit-learn**. В качестве набора данных для обучения модели используется [Spambase Dataset](https://archive.ics.uci.edu/ml/datasets/spambase). Набор данных содержит 5574 размеченных сообщения ("**spam**" и "**ham**"). 

Альтернативная [ссылка](https://drive.google.com/file/d/1k6oeLNi2hki_FcfLg06XQI1miZzgU-0d/view?usp=sharing) на набор данных.

Скачиваем набор данных и загружаем **spambase_dataset.csv** сюда.

In [5]:
pip install googledrivedownloader

In [9]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1k6oeLNi2hki_FcfLg06XQI1miZzgU-0d', dest_path='./spambase_dataset.csv')

In [16]:
import pandas as pd
df = pd.read_csv('spambase_dataset.csv', encoding='latin-1')

Оставляем только нужные столбцы: текст сообщения (**v2**) и метка (**v1**).

In [17]:
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Удаляем дублирующиеся тексты:

In [18]:
df = df.drop_duplicates('text')

Заменяем метки на бинарные:

In [19]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

Теперь с текстом нужно произвести несколько операций. В первую очередь из текста нужно удалить знаки препинания (**re.sub()**). Затем, текст нужно привести к нижнему регистру (**str.lower()**). Разбить текст на **токены**. Также необходимо избавиться от **стоп-слов**, которые можно найти в **nltk.corpus.stopwords**. И, наконец, нормализировать текст при помощи стеммера **Snowball**.

Примеры использования стеммеров можно найти по [ссылке](https://www.nltk.org/howto/stem.html).

In [20]:
from nltk import stem
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = [i for i in text.split() if not i in stopwords]
    text = [stemmer.stem(i) for i in text]
    return " ".join([i for i in text])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Проверка, что функция работает верно.

In [21]:
assert preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.") == "im gonna home soon dont want talk stuff anymor tonight k ive cri enough today"
assert preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...") == "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"

Применяем получившуюся функцию к текстам.

In [22]:
df['text'] = df['text'].apply(preprocess)

Разделение данных на обучающую и тестовую выборки.

In [23]:
y = df['label'].values

Теперь нужно разделить данные на тестовую (**test**) и обучающую (**train**) выборки. Сделать это можно с помощью **train_test_split()** библиотеки **scikit-learn**.

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.25, random_state=94)

Перейдём к обучению классификатора.

Начнём с извлечения признаков при помощи **TfidfVectorizer()**. 

Подробнее о различных способах представления текстов можно прочитать по [ссылке](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

Теперь обучим классификатор **SVM**.

In [28]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

model = LinearSVC(random_state = 94, C = 1.3)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

Посмотрим на результаты оценки получившейся модели.

In [29]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.980     0.996     0.988      1133
           1      0.965     0.856     0.907       160

    accuracy                          0.978      1293
   macro avg      0.972     0.926     0.948      1293
weighted avg      0.978     0.978     0.978      1293



Теперь можно выполнять предсказание для конкретного текста.

In [30]:
txt = "Take your prize, more than 100 computers, smartphones and TVs are supposed to be played in a free quiz. Call by phone 8 800 243 456"
txt = preprocess(txt)
txt = vectorizer.transform([txt])

In [31]:
model.predict(txt)

array([1])

*   array([0]) - HAM
*   array([1]) - SPAM

